In [ ]:
from climateforcing.utils import mkdir_p
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pickle
import warnings

In [ ]:
IIRF_HORIZON=100
IIRF_MAX = 99.95
GAS_BOX_AXIS=4
TIME_AXIS=0
M_ATMOS = 5.1352e18 # mass of atmosphere, kg

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']
n_scenarios = len(scenarios)

In [ ]:
timebounds = np.arange(1750, 2501.5, 1)
timesteps  = np.arange(1750.5, 2501, 1)
timestep   = 1

In [ ]:
n_timesteps = len(timesteps)
n_timebounds = len(timebounds)

In [ ]:
gas_list = [
    "C2F6",
    "C3F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "cC4F8",  # not standard PubChem but used extensively in AR6
    "CCl4",
    "CF4",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "CFC-11",
    "CFC-12",
    "CH2Cl2",
    "CH3Br",
    "CH3CCl3",
    "CH3Cl",
    "CH4",
    "CHCl3",
    "CO2",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "HCFC-141b",
    "HCFC-142b",
    "HCFC-22",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
    "N2O",
    "NF3",
    "SF6",
    "SO2F2",
]

n_gases = len(gas_list)

In [ ]:
# grab some emissions
emissions_array = np.ones((751, n_scenarios, n_gases, 1, 1)) * np.nan
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    for igas, gas in enumerate(gas_list):
        gas_rcmip_name = gas.replace("-", "")
        emissions_array[:, iscen, igas, 0, 0] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).squeeze().values.T
    
        # CO2 and N2O units need to behave
        if gas in ('CO2', 'N2O'):
            emissions_array[:, iscen, igas, ...] = emissions_array[:, iscen, igas, ...] / 1000

In [ ]:
emissions_array[-1, 7, :, 0, 0]

In [ ]:
# grab some concentrations
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for scenario in scenarios:
    concentration_rcmip[scenario] = {}
    for gas in gas_list:
        gas_rcmip_name = gas.replace("-", "")
        concentration_rcmip[scenario][gas] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.squeeze()

In [ ]:
#concentration_rcmip['ssp119']['CO2']

In [ ]:
# grab indicative temperature projections for feedbacks
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

```
Emissions are          1750.5, 1751.5, 1752.5, ..., 2500.5  (751 array)
Concentrations are 1750.0, 1751.0, 1752.0, ...,   ..., 2501.0 (752 array)  
```

In [ ]:
(emissions_array.shape[0]+1,) + emissions_array.shape[1:]

In [ ]:
concentration_array = np.ones((emissions_array.shape[0]+1,) + emissions_array.shape[1:]) * np.nan

In [ ]:
for iscen, scenario in enumerate(scenarios):
    for igas, gas in enumerate(gas_list):
        concentration_array[0, iscen, igas, ...] = concentration_rcmip[scenario][gas][0]

In [ ]:
#concentration_array[0, ..., 0, 0]

In [ ]:
N_GAS_BOXES=4

partition_fraction = {}
for gas in gas_list:
    partition_fraction[gas] = np.zeros(N_GAS_BOXES)
    partition_fraction[gas][0] = 1
partition_fraction["CO2"] = np.array([0.2173, 0.2240, 0.2824, 0.2763])

lifetime = {
    "C2F6": 10000,
    "C3F8": 2600,
    "C4F10": 2600,
    "C5F12": 4100,
    "C6F14": 3100,
    "C7F16": 3000,
    "C8F18": 3000,
    "cC4F8": 3200,  # not standard PubChem name but used extensively in AR6
    "CCl4": 32,
    "CF4": 50000,
    "CFC-113": 93,
    "CFC-114": 189,
    "CFC-115": 540,
    "CFC-11": 52,
    "CFC-12": 102,
    "CH2Cl2": 0.493,
    "CH3Br": 0.8,
    "CH3CCl3": 5,
    "CH3Cl": 0.9,
    "CH4": 8.25,  # atmospheric burden lifetime in pre-industrial conditions. Source: Leach et al. (2021)
    "CHCl3": 0.501,
    "CO2": np.array([1e9, 394.4, 36.54, 4.304]),
    "Halon-1211": 16,
    "Halon-1301": 72,
    "Halon-2402": 28,
    "HCFC-141b": 9.4,
    "HCFC-142b": 18,
    "HCFC-22": 11.9,
    "HFC-125": 30,
    "HFC-134a": 14,
    "HFC-143a": 51,
    "HFC-152a": 1.6,
    "HFC-227ea": 36,
    "HFC-23": 228,
    "HFC-236fa": 213,
    "HFC-245fa": 7.9,
    "HFC-32": 5.4,
    "HFC-365mfc": 8.9,
    "HFC-4310mee": 17,
    "N2O": 123, #109,
    "NF3": 569,
    "SF6": 3200,
    "SO2F2": 36,
}

MOLWT = {
    "AIR": 28.97,  # reference?
    "C": 12.011,
    "C2F6": 138.01,
    "C3F8": 188.02,
    "C4F10": 238.03,
    "C5F12": 288.03,
    "C6F14": 338.04,
    "C7F16": 388.05,
    "C8F18": 438.06,
    "cC4F8": 200.03,  # not standard PubChem but used extensively in AR6
    "CCl4": 153.8,
    "CF4": 88.004,
    "CFC-113": 187.37,
    "CFC-114": 170.92,
    "CFC-115": 154.46,
    "CFC-11": 137.36,
    "CFC-12": 120.91,
    "CH2Cl2": 84.93,
    "CH3Br": 94.94,
    "CH3CCl3": 133.4,
    "CH3Cl": 50.49,
    "CH4": 16.043,
    "CHCl3": 119.37,
    "CO2": 44.009,
    "Halon-1211": 165.36,
    "Halon-1301": 148.91,
    "Halon-2402": 259.82,
    "HCFC-141b": 116.95,
    "HCFC-142b": 100.49,
    "HCFC-22": 86.47,
    "HFC-125": 120.02,
    "HFC-134a": 102.03,
    "HFC-143a": 84.04,
    "HFC-152a": 66.05,
    "HFC-227ea": 170.03,
    "HFC-23": 70.014,
    "HFC-236fa": 152.04,
    "HFC-245fa": 134.05,
    "HFC-32": 52.023,
    "HFC-365mfc": 148.07,
    "HFC-4310mee": 252.05,
    "N": 14.007,
    "N2": 28.014,
    "N2O": 44.013,
    "NF3": 71.002,
    "NO": 30.006,
    "NO2": 46.006,
    "S": 32.07,
    "SF6": 146.06,
    "SO2": 64.069,
    "SO2F2": 102.06,
}

iirf_0 = {gas: (lifetime[gas] * (1 - np.exp(-IIRF_HORIZON / lifetime[gas]))) for gas in gas_list}
iirf_0['CO2'] = 29

iirf_uptake = {gas: 0 for gas in gas_list}
iirf_uptake["CO2"] = 0.00846

iirf_temperature = {gas: 0 for gas in gas_list}
#iirf_temperature = {gas: -iirf_0[gas]*0.015 for gas in gas_list}
iirf_temperature["CO2"] = 4.0
iirf_temperature["CH4"] = -0.3

iirf_airborne = {gas: 0 for gas in gas_list}
iirf_airborne["CO2"] = 0.000819
iirf_airborne["CH4"] = 0.00032
iirf_airborne["N2O"] = -0.0065    # hand tuning to PD and PI lifetimes: Prather et al. 2015

concentration_per_emission = {}
for gas in gas_list:
    concentration_per_emission[gas] = (
        1 / (M_ATMOS / 1e18 * MOLWT[gas] / MOLWT["AIR"])
    )

rcmip_emissions_adjustment = {}
for gas in gas_list:
    rcmip_emissions_adjustment[gas] = 0
rcmip_emissions_adjustment.update(
    {
        "CF4": 0.010071225,
        "CCl4": 0.024856862,
        "CH2Cl2": 246.6579,
        "CH3Br": 105.08773,
        "CH3Cl": 4275.7449,
        "CHCl3": 300.92479,
        "Halon-1211": 0.0077232726,
    }
)

In [ ]:
iirf_0["N2O"]

In [ ]:
lifetime_array = np.ones((1, 1, n_gases, 1, N_GAS_BOXES)) * np.nan
partition_fraction_array = np.ones((1, 1, n_gases, 1, N_GAS_BOXES)) * np.nan
iirf_0_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
iirf_airborne_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
iirf_temperature_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
iirf_uptake_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
concentration_per_emission_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan
rcmip_emissions_adjustment_array = np.ones((1, 1, n_gases, 1, 1)) * np.nan  # could be time dependent

baseline_concentration_array = concentration_array[0:1, :, :, :, :]

# we would also loop/parallel over config here
for igas, gas in enumerate(gas_list):
    lifetime_array[0, 0, igas, 0, :] = lifetime[gas]
    partition_fraction_array[0, 0, igas, 0, :] = partition_fraction[gas]
    iirf_0_array[0, 0, igas, 0, :] = iirf_0[gas]
    iirf_airborne_array[0, 0, igas, 0, :] = iirf_airborne[gas]
    iirf_temperature_array[0, 0, igas, 0, :] = iirf_temperature[gas]
    iirf_uptake_array[0, 0, igas, 0, :] = iirf_uptake[gas]
    concentration_per_emission_array[0, 0, igas, 0, 0] = concentration_per_emission[gas]
    rcmip_emissions_adjustment_array[0, 0, igas, 0, 0] = rcmip_emissions_adjustment[gas]

In [ ]:
baseline_concentration_array.shape

In [ ]:
g1 = np.sum(
    partition_fraction_array * lifetime_array *
    (1 - (1 + IIRF_HORIZON/lifetime_array) *
    np.exp(-IIRF_HORIZON/lifetime_array)), axis=4, keepdims=True
)

g0 = np.exp(
    -1 * np.sum(
        partition_fraction_array*lifetime_array*(1 - np.exp(-IIRF_HORIZON/lifetime_array)), axis=4, keepdims=True
    )/g1
)

In [ ]:
g1[0,0,21,0,0], g0[0,0,21,0,0]  # looks right for CO2

In [ ]:
gas_list[21]

In [ ]:
# we need separate baselines for both the forcing and concentrations at pre-industrial.

alpha_lifetime_array = np.ones((n_timesteps, n_scenarios, n_gases, 1, 1)) * np.nan
airborne_emissions_array = np.ones((n_timebounds, n_scenarios, n_gases, 1, 1)) * np.nan
gas_boxes_array = np.zeros((1, n_scenarios, n_gases, 1, N_GAS_BOXES))  # initial condition for restarts
#baseline_cumulative_emissions_array = np.zeros((1, n_scenarios, n_gases, 1, 1))
#cumulative_emissions_array = np.ones((n_timebounds, n_scenarios, n_gases, 1, 1)) * np.nan

#cumulative_emissions_array[0, ...] = 0
cumulative_emissions_array = np.ones((n_timebounds, n_scenarios, n_gases, 1, 1)) * np.nan
cumulative_emissions_array[0, ...] = 0  # initial condition: make interface option
cumulative_emissions_array[1:, ...] = np.cumsum(emissions_array * timestep, axis=TIME_AXIS)
#natural_emissions_adjustment_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan

airborne_emissions_array[0, ...] = 0  # initial condition: make interface option

In [ ]:
def calculate_alpha(
    cumulative_emissions,
    airborne_emissions,
    temperature,
    iirf_0,
    iirf_uptake,
    iirf_temperature,
    iirf_airborne,
    g0,
    g1,
    iirf_max,
):

    iirf = iirf_0 + iirf_uptake * (cumulative_emissions-airborne_emissions) + iirf_temperature * temperature + iirf_airborne * airborne_emissions
    iirf = (iirf>iirf_max) * iirf_max + iirf * (iirf<iirf_max)

    # overflow and invalid value errors occur with very large and small values
    # in the exponential. This happens with very long lifetime GHGs. Usually
    # these GHGs don't have a temperature dependence on IIRF but even if they
    # did the lifetimes are so long that it is unlikely to have an effect.
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        alpha = g0 * np.exp(iirf / g1)
        alpha[np.isnan(alpha)]=1
    return alpha

In [ ]:
def step_concentration(
    emissions,
    gas_boxes_old,
    airborne_emissions_old,
    concentration_per_emission,
    lifetime,
    alpha_lifetime,
    partition_fraction,
    baseline_concentration,
    timestep=1,
    natural_emissions_adjustment=0,
):

#    # this is needed because we set alphas to one and not nan, so non-GHGs
#    # cause a warning.
#    with warnings.catch_warnings():
#        warnings.simplefilter('ignore')
    decay_rate = timestep/(alpha_lifetime * lifetime)
    decay_factor = np.exp(-decay_rate)
    gas_boxes_new = (
        partition_fraction *
        (emissions-natural_emissions_adjustment) *
        1 / decay_rate *
        (1 - decay_factor) * timestep + gas_boxes_old * decay_factor
    )
    airborne_emissions_new = np.sum(gas_boxes_new, axis=GAS_BOX_AXIS, keepdims=True)
    concentration_out = baseline_concentration + concentration_per_emission * airborne_emissions_new

    return concentration_out, gas_boxes_new, airborne_emissions_new

In [ ]:
gas_boxes_array = np.zeros((1, n_scenarios, n_gases, 1, N_GAS_BOXES))  # initial condition for restarts

for i_timestep in range(n_timesteps):  # 0 to 751
    alpha_lifetime_array[i_timestep, ...] = calculate_alpha(   # this timestep
        cumulative_emissions_array[i_timestep, ...],  # last timebound
        airborne_emissions_array[i_timestep, ...],  # last timebound
        ssp245_temperature_rfmip[i_timestep],  # last timebound
        iirf_0_array,
        iirf_uptake_array,
        iirf_temperature_array,
        iirf_airborne_array,
        g0,
        g1,
        IIRF_MAX
    )
    concentration_array[i_timestep+1, ...], gas_boxes_array, airborne_emissions_array[i_timestep+1, ...] = step_concentration( # next timebound
        emissions_array[i_timestep, ...],  # this timestep
        gas_boxes_array, # last timebound
        airborne_emissions_array[i_timestep, ...],  # last timebound
        concentration_per_emission_array,
        lifetime_array,
        alpha_lifetime_array[i_timestep, ...],
        partition_fraction_array,
        baseline_concentration_array,
        timestep,
        rcmip_emissions_adjustment_array,
    )

In [ ]:
pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 21, 0, 0])
pl.plot(np.arange(1750.5, 2022), concentration_rcmip['ssp245']['CO2'][:272])

In [ ]:
pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 39, 0, 0])
pl.plot(np.arange(1750.5, 2022), concentration_rcmip['ssp245']['N2O'][:272])

In [ ]:
for iscen, scenario in enumerate(scenarios):
    pl.plot(timebounds, concentration_array[:,iscen,21,0,0], label=scenario)
pl.legend();

In [ ]:
concentration = {}

for iscen, scenario in enumerate(scenarios):
    concentration[scenario] = {}
    for igas, gas in enumerate(gas_list):
        concentration[scenario][gas] = concentration_array[:, iscen, igas, 0, 0]

In [ ]:
for iscen, scenario in enumerate(scenarios):
    fig, ax = pl.subplots(6, 8, figsize=(16,16))
    for igas, gas in enumerate(gas_list):
        iy = igas % 8
        ix = igas // 8
        ax[ix, iy].plot(timebounds, concentration[scenario][gas], label='FaIR 2.1', color='k')
        ax[ix, iy].plot(timebounds[:-1], concentration_rcmip[scenario][gas], label='History + MAGICC6', color='r')
        ax[ix, iy].set_title(gas)
    fig.tight_layout()

In [ ]:
alpha_lifetime_array[:, 0, 39, 0, 0] * lifetime_array[0, 0, 39, 0, 0]

In [ ]:
alpha_lifetime_array[258, 0, 39, 0, 0] * lifetime_array[0, 0, 39, 0, 0]  # target 109

In [ ]:
alpha_lifetime_array[100, 0, 39, 0, 0] * lifetime_array[0, 0, 39, 0, 0]  # target 123

In [ ]:
mkdir_p("../data/output/")
with open("../data/output/rcmip-fair21-concentrations.pkl","wb") as fileout:
    pickle.dump(concentration, fileout)